In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [ ]:
data_path = r"ODIR-5K\data.xlsx"
df = pd.read_excel(data_path)
df.head(10)

In [ ]:
# label columns and combining left and right fundus image

label_cols = ['N','D','G','C','A','H','M','O']

left_df = df[['Left-Fundus']+label_cols].copy()
left_df.rename(columns={'Left-Fundus':'image'}, inplace=True)

right_df = df[['Right-Fundus']+label_cols].copy()
right_df.rename(columns={'Right-Fundus':'image'}, inplace=True)

# Combine into one dataframe
full_df = pd.concat([left_df, right_df], axis=0).reset_index(drop=True)
full_df = full_df.dropna(subset=['image'])

In [ ]:
# save CSV
full_df.to_csv("ODIR-5K/full_df.csv", index=False)
full_df.head(10) 

In [ ]:
# Load full_df.csv
df = pd.read_csv("ODIR-5K/full_df.csv")

# Input and output directories
input_dir = "ODIR-5K/Training Images"
output_dir = "ODIR-5K/preprocessed_images"
os.makedirs(output_dir, exist_ok=True)

# CLAHE function
def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    merged = cv2.merge((cl, a, b))
    return cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

# Apply CLAHE to all images in full_df
for img_name in df['image'].unique():
    in_path = os.path.join(input_dir, img_name)
    out_path = os.path.join(output_dir, img_name)

    if os.path.exists(in_path) and not os.path.exists(out_path):
        img = cv2.imread(in_path)
        if img is not None:
            enhanced = apply_clahe(img)
            cv2.imwrite(out_path, enhanced)

In [ ]:
import random
import cv2
import matplotlib.pyplot as plt

# Folder paths
original_path = "ODIR-5K/Training Images"
processed_path = "ODIR-5K/preprocessed_images"

# Pick 2 random images
images = random.sample(os.listdir(processed_path), 2)

for img_name in images:
    orig = cv2.imread(f"{original_path}/{img_name}")
    proc = cv2.imread(f"{processed_path}/{img_name}")

    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
    proc = cv2.cvtColor(proc, cv2.COLOR_BGR2RGB)

    # Show both
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 2, 1)
    plt.imshow(orig)
    plt.title("Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(proc)
    plt.title("CLAHE Enhanced")
    plt.axis('off')

    plt.suptitle(img_name)
    plt.show()

In [ ]:
label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

In [ ]:
class RetinalDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.image_dir, row['image'])

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        labels = torch.tensor(row[label_cols].values.astype(np.float32))
        return image, labels


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


In [ ]:
#  creating Dataset and DataLoader

dataset = RetinalDataset(
    csv_file="ODIR-5K/full_df.csv",
    image_dir="ODIR-5K/preprocessed_images",
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

images, labels = next(iter(dataloader))
print("Image batch shape:", images.shape)
print("Label batch shape:", labels.shape)


In [ ]:
import timm
import torch.nn as nn

class ViTModel(nn.Module):
    def __init__(self, model_name='vit_base_patch16_224', num_classes=8):
        super(ViTModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True)
        in_features = self.model.head.in_features
        self.model.head = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ViTModel()
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm

num_epochs = 5  # You can increase this later

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    loop = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()  # for BCEWithLogitsLoss

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}, Average Loss: {running_loss / len(dataloader):.4f}")
